# A. Document preprocessing and representation

## Load data

In [24]:
#!pip install pyLDAvis

In [25]:
import pandas as pd
import numpy as np
from seaborn import heatmap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from gensim import corpora, models
from pyLDAvis.gensim_models import prepare
import pickle
import pyLDAvis
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import requests
import json

ModuleNotFoundError: ignored

In [ ]:
import pickle
import urllib.request

In [ ]:
import urllib.request

In [ ]:
url = "https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json"
response = urllib.request.urlopen(url)

In [ ]:
import json
data = json.load(response)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
path = '/content/drive/MyDrive/Group 1/Stemmed.pkl' #TK's path
#path = '/content/drive/MyDrive/Group 1 - Text Analytics/Class/Datasets/Stemmed.pkl'
with open(path, 'rb') as file:
  stemmed = pickle.load(file)

data_url = 'https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json'
data = pd.read_json(data_url)



In [ ]:
required_topics = ['soc.religion.christian', 'rec.sport.hockey', 'talk.politics.mideast', 'rec.motorcycles']
se = pd.Series(stemmed)
data['preprocessed'] = se.values

# Filter data for required topics
data = data[data['target_names'].isin(required_topics)]

In [ ]:
data

## Preprocessing

In [ ]:
# Apply tf-idf transformer to the dataset
vectorizer = TfidfVectorizer(max_df=0.7, min_df=0.1)
data_tfidf = vectorizer.fit_transform(data['preprocessed'])

# Store feature names in a list
words = vectorizer.get_feature_names_out()

# B. K-means clustering

## Clustering

In [ ]:
kmeans = KMeans(n_clusters = 4, max_iter=1000, random_state=42)
kmeans.fit(data_tfidf)

## Cluster labeling

Here we cluster the dataset into 4 clusters with a seed 42

In [ ]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
print(common_words)

calculates the cluster centers for the K-means clustering model (kmeans) and sorts them in ascending order. The argsort() function returns the indices that would sort the array in ascending order. Here, the argsort() is performed on the cluster_centers_ attribute of the K-means model. The slicing [:,-1:-11:-1] selects the last 10 indices of each cluster center in descending order

In [ ]:
for num, centroid in enumerate(common_words):
 print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

code then enters a loop using for num, centroid in enumerate(common_words):. This loop iterates over each cluster center in the common_words array.

Within the loop, the line print(str(num) + ' : ' + ', '.join(words[word] for word in centroid)) prints the cluster number (num) along with the corresponding target names. Here, words is likely a dictionary or a list that maps indices to target names. The join() function is used to concatenate the target names corresponding to the indices in the centroid array, separated by commas

B 5. Add a column to the data frame called ‘cluster’ using kmeans.labels_. What is it
doing?

In [ ]:
# Add the 'cluster' column to the DataFrame
data['cluster'] = kmeans.labels_

In [ ]:
data

## Heatmap

B 6

In [ ]:
clusters = data.groupby(['cluster', 'target_names']).size()
fig, ax1 = plt.subplots(figsize = (10, 7))
heatmap(clusters.unstack(level = 'target_names'), ax = ax1, cmap = 'Reds')
ax1.set_xlabel('target_names').set_size(8)
ax1.set_ylabel('cluster').set_size(8)

B 7

Based on question 4, we have:

0 : edu, not, israel, write, on, peopl, articl, would, new, know -> talk.politics.mideast (17)

1 : team, game, plai, hockei, year, edu, not, get, would, write -> rec.sport.hockey (10)

2 : bike, com, dod, articl, write, edu, get, like, apr, not -> rec.motorcycles (8)

3 : god, christian, not, believ, would, peopl, on, sai, edu, think -> soc.religion.christian (15)

Therefore, we would map cluster 0 to 17, 1 to 10, 2 to 8 and 3 to 15.

In [ ]:
# Define your mapping as a dictionary
mapping = {0: 17, 1: 10, 2: 8, 3: 15}

# Use the map function to apply the mapping to the 'cluster' column
data['pred'] = data['cluster'].map(mapping)

## Classification Report

In [ ]:
e
y_true = data[from sklearn.metrics import classification_report
# Extract the true target values from the DataFram'target']

# Extract the predicted values from the DataFrame
y_pred = data['pred']

# Print the classification report
print(classification_report(y_true, y_pred))

Precision values range from 0.48 to 0.98. The precision values are relatively high, so the model makes accurate positive predictions for most classes

Recall values range from 0.65 to 0.91. The recall scores indicate that the model is able to identify a substantial portion of the positive instances for most classes

F1-scores range from 0.63 to 0.82. The F1-scores provide a balanced measure of precision and recall, and they show reasonable performance for most classes

Support values indicate the number of instances for each class. Higher support values enhance the reliability of the metrics

Overall accuracy of the model is 0.73, approximately 73% of the instances are correctly classified

# C. LDA topic modelling

In [ ]:
df

In [ ]:
# Recall df
required_topics = ['soc.religion.christian', 'rec.sport.hockey', 'talk.politics.mideast', 'rec.motorcycles']
se = pd.Series(stemmed)
df['preprocessed'] = se.values

# Filter data for required topics
df = df[df['target_names'].isin(required_topics)]

In [ ]:
from gensim import corpora

# Split the texts in 'content column
documents = df['preprocessed'].apply(lambda x: x.split())

# Create a gensim dictionary from the documents
dictionary = corpora.Dictionary(documents)

# Filter out words that occur less than 118 documents, or more than 95% of the documents
dictionary.filter_extremes(no_below=118, no_above=0.95)

# Derive the absolute frequency matrix using gensim’s doc2bow
corpus = [dictionary.doc2bow(doc) for doc in documents]


## LDA

In [ ]:
lda = models.LdaModel(corpus, num_topics=4, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)

`lda.show_topics()` shows top words from each topic.

In each tuple, the first value is the topic ID (0, 1, 2, 3).

The second value is a string representing the top words in that topic along with their probabilities. The number before each word is the estimated probability that the word belongs to this topic. For instance, "I" has a 6.2% estimated probability of belonging to topic 0, ">" has a 4.5% probability, and so on.



In [ ]:
lda.show_topics()

In [ ]:
topics=lda[corpus]
df['topics_lda']= [max(topics[i],key=lambda item:item[1])[0] for i in range(len(topics))]

In [ ]:
df

## Heatmap

In [ ]:
clusters = df.groupby(['topics_lda', 'target_names']).size()
fig, ax1 = plt.subplots(figsize = (10, 7))
heatmap(clusters.unstack(level = 'target_names'), ax = ax1, cmap = 'Reds')
ax1.set_xlabel('target_names').set_size(8)
ax1.set_ylabel('topics_lda').set_size(8)

## Cluster Labeling

Based on question lda_topics, we have:

0 : soc.religion.christian (15)

1 : talk.politics.mideast (17) 

2 : rec.sport.hockey (10) 

3 : rec.motorcycles (8)

Therefore, we would map cluster 0 to 15, 1 to 17, 2 to 10 and 3 to 8.

In [ ]:
# Define your mapping as a dictionary
mapping = {0: 15, 1: 17, 2: 10, 3: 8}

# Use the map function to apply the mapping to the 'cluster' column
df['pred'] = df['topics_lda'].map(mapping)

In [ ]:
df

## Classification Report

In [ ]:
y_true = df['target']

# Extract the predicted values from the DataFrame
y_pred = df['pred']

# Print the classification report
print(classification_report(y_true, y_pred))

*Classification Report C vs. B*

Precision values range from 0.29 to 0.96. The precision scores for several classes have significantly decreased, suggesting a higher number of false positive predictions

Recall values range from 0.09 to 0.92. The recall scores have also changed, with some classes showing a decrease in the ability to identify positive instances correctly

F1-scores range from 0.14 to 0.72. The F1-scores have dropped, indicating a decrease in the overall balance between precision and recall

The support values remain the same, indicating the number of instances for each class

Overall accuracy of the model is reported as 0.50, approximately 50% of the instances are correctly classified

In [ ]:
topics=lda[corpus]
df['topics_lda']= [max(topics[i],key=lambda item:item[1])[0] for i in
range(len(topics))]

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(lda, corpus, dictionary)
LDAvis_prepared